In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
import os
project_path = "/content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation"
os.chdir(project_path)
print("Current directory:", os.getcwd())


Current directory: /content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation


In [7]:
!pip install --upgrade --quiet torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 \
     -f https://download.pytorch.org/whl/torch_stable.html

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 119.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 834.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [8]:
!pip install -r requirements.txt

  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached MarkupSafe-2.1.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached numpy-2.1.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached pandas-2.3.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
  Using cached pillow-11.0.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (9.1 kB)
  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached regex-2025.7.34-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (40 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cac

In [1]:
import os

project_path = "/content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation"

# Remove old links if they exist (ignore errors if they don’t)
!rm -rf /content/data /content/src

# Make new symbolic links
!ln -s "$project_path/data" /content/data
!ln -s "$project_path/src"  /content/src

# Verify
!ls /content/data | head -n 10
!ls /content/src


images
ISIC_2020_Training_GroundTruth.csv
ISIC_2020_Training_Metadata_v2.csv
dataset.py  main.py  model.py  __pycache__  train.py


In [2]:
import os, sys
sys.path.append("/content/src")   # lets you `import dataset`, `import model`, etc.
os.chdir("/content/src")
print("Now inside src folder:", os.getcwd())


Now inside src folder: /content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation/src


In [3]:
from transformers import BertTokenizer
from torchvision import transforms
from dataset import ISICDataset

# Paths inside Colab
IMAGE_DIR = "/content/data/images/train"
METADATA_CSV = "/content/data/ISIC_2020_Training_Metadata_v2.csv"
GROUNDTRUTH_CSV = "/content/data/ISIC_2020_Training_GroundTruth.csv"

# Prepare tokenizer and image transform
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor()
])

# Create the dataset limited to 200 samples
dataset = ISICDataset(
    IMAGE_DIR,
    METADATA_CSV,
    GROUNDTRUTH_CSV,
    tokenizer,
    transform,
    limit_samples=200      # ← keeps only 200 images
)

print("Dataset length =", len(dataset))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Dataset limited to 200 samples
Final merged columns: ['image_name', 'patient_id_x', 'sex_x', 'age_approx_x', 'anatom_site_general_challenge_x', 'diagnosis_x', 'benign_malignant_x', 'target_x', 'patient_id_y', 'lesion_id', 'sex_y', 'age_approx_y', 'anatom_site_general_challenge_y', 'diagnosis_y', 'benign_malignant_y', 'target_y']
Final dataset size: 200 samples
Dataset length = 200


In [4]:
!python train.py


/usr/local/lib/python3.11/dist-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-08-07 14:06:30.770041: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754575591.060352   10233 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754575591.133520   10233 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754575591.686231   10233 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754575591.686274   10233 computation_pl

In [6]:
!grep -n "torch.save" -n /content/src/train.py


grep: /content/src/train.py: No such file or directory


In [7]:
import os, glob
print("Current working dir:", os.getcwd())
print("Any src folders we can see?:", glob.glob("/content/**/src", recursive=True))


Current working dir: /content
Any src folders we can see?: []


In [12]:
# Unmount and clean up the old mount
!fusermount -u /content/drive
!rm -rf /content/drive          # remove leftover files

# Now mount fresh
from google.colab import drive
drive.mount('/content/drive')   # no force_remount needed


fusermount: failed to unmount /content/drive: Invalid argument
Mounted at /content/drive


In [13]:
import os, shutil, sys, glob

project = "/content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation"

# Remove any old links (ignore errors)
shutil.rmtree("/content/data", ignore_errors=True)
shutil.rmtree("/content/src",  ignore_errors=True)

# Make fresh links
os.symlink(f"{project}/data", "/content/data")
os.symlink(f"{project}/src",  "/content/src")

# Switch into src and add it to the import path
sys.path.append("/content/src")
os.chdir("/content/src")
print("Now inside:", os.getcwd())
print("Files here:", glob.glob('*'))


Now inside: /content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation/src
Files here: ['main.py', '__pycache__', 'model.py', 'dataset.py', 'train.py']


In [14]:
import os, glob

# Confirm we are in the right place
print("Working directory:", os.getcwd())

# List a few expected files
print("Files inside /content/src:")
print(glob.glob('/content/src/*')[:10])     # first 10 entries


Working directory: /content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation/src
Files inside /content/src:
['/content/src/main.py', '/content/src/__pycache__', '/content/src/model.py', '/content/src/dataset.py', '/content/src/train.py']


In [16]:
!pip install --quiet python-dotenv==1.1.1



In [17]:
!python train.py


/usr/local/lib/python3.11/dist-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-08-07 14:59:56.457918: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754578796.491842    5093 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754578796.502505    5093 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754578796.529046    5093 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754578796.529078    5093 computation_pl

In [18]:
from google.colab import drive, runtime
# Unmount (ignore errors), then mount fresh
!fusermount -u /content/drive || true
drive.mount('/content/drive')

# Re-establish shortcuts
import os, shutil, sys
proj = "/content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation"
for link in ["/content/data", "/content/src"]:
    shutil.rmtree(link, ignore_errors=True)
os.symlink(f"{proj}/data", "/content/data")
os.symlink(f"{proj}/src",  "/content/src")
sys.path.append("/content/src")


fusermount: failed to unmount /content/drive: Device or resource busy
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileExistsError: [Errno 17] File exists: '/content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation/data' -> '/content/data'

In [19]:
import os, shutil

for link in ["/content/data", "/content/src"]:
    if os.path.islink(link) or os.path.exists(link):
        shutil.rmtree(link, ignore_errors=True)
print("Old links cleared")


Old links cleared


In [20]:
from pathlib import Path
import sys, os, glob

proj = Path("/content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation")

assert proj.exists(), "❌ Project folder not found in Drive"

assert (proj / "data").exists(), "❌ data folder missing in project"
assert (proj / "src").exists(),  "❌ src folder missing in project"

# recreate fresh links
os.symlink(proj / "data", "/content/data")
os.symlink(proj / "src",  "/content/src")

# activate the source folder
sys.path.append("/content/src")
os.chdir("/content/src")
print("✅ Links rebuilt. Now inside:", os.getcwd())
print("Files:", glob.glob('*')[:10])


FileExistsError: [Errno 17] File exists: '/content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation/data' -> '/content/data'

In [21]:
import os, shutil, sys, glob
from pathlib import Path

# 1️⃣  Remove OLD links if they are still present
for link in ("/content/data", "/content/src"):
    if os.path.islink(link) or os.path.exists(link):
        shutil.rmtree(link, ignore_errors=True)

# 2️⃣  Make sure Drive is mounted
from google.colab import drive
drive.mount("/content/drive", force_remount=True)   # will prompt for auth once

# 3️⃣  Define the project folder inside Drive
proj = Path("/content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation")
assert proj.exists(), "❌ Project folder not found—check the path and spelling."

# 4️⃣  Re-create fresh links
os.symlink(proj / "data", "/content/data")
os.symlink(proj / "src",  "/content/src")

# 5️⃣  Switch into src and add it to the Python path
sys.path.append("/content/src")
os.chdir("/content/src")

print("✅ Links rebuilt. Now inside:", os.getcwd())
print("Files:", glob.glob('*'))


Mounted at /content/drive


FileExistsError: [Errno 17] File exists: '/content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation/data' -> '/content/data'

In [22]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)   # remount, prompts once

import os, glob
print("Top-level folders in MyDrive:", glob.glob("/content/drive/MyDrive/*"))


Mounted at /content/drive
Top-level folders in MyDrive: ['/content/drive/MyDrive/AKSHAT SINHA NEW RESUME-HALF (1).pdf', '/content/drive/MyDrive/Colab Notebooks', '/content/drive/MyDrive/DRIVEGOOD_VIDEO_DEMO_merged.mp4', '/content/drive/MyDrive/DRIVEGOOD_VIDEO_DEMO_merged_merged.mp4', '/content/drive/MyDrive/VIT -Internship - Project Presentation -mern.pptx', '/content/drive/MyDrive/2 NEO PAT RESUME 22BCE2218 AKSHAT SINHA (2) (1).pdf', '/content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation', '/content/drive/MyDrive/CVReview1.docx']


In [23]:
import os, shutil, sys, glob
from pathlib import Path

# 1 – absolute path to the project in Drive
proj = Path("/content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation")

# 2 – delete old links if they exist
for link in ("/content/data", "/content/src"):
    if os.path.islink(link) or os.path.exists(link):
        shutil.rmtree(link, ignore_errors=True)

# 3 – sanity-check the real folders before linking
assert (proj / "data").exists(),  "❌ data/ sub-folder missing in project"
assert (proj / "src").exists(),   "❌ src/ sub-folder missing in project"

# 4 – make fresh links
os.symlink(proj / "data", "/content/data")
os.symlink(proj / "src",  "/content/src")

# 5 – activate src
sys.path.append("/content/src")
os.chdir("/content/src")

print("✅ Links rebuilt. Now inside:", os.getcwd())
print("Files here:", glob.glob('*'))


FileExistsError: [Errno 17] File exists: '/content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation/data' -> '/content/data'

In [24]:
import os, shutil

for link in ("/content/data", "/content/src"):
    # unlink if it’s a symlink; otherwise remove the whole folder
    if os.path.islink(link):
        os.unlink(link)
    elif os.path.exists(link):
        shutil.rmtree(link, ignore_errors=True)

print("✅ Old /content/data and /content/src cleared")


✅ Old /content/data and /content/src cleared


In [25]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)   # will prompt once

import os, sys, glob
from pathlib import Path

proj = Path("/content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation")

assert proj.exists(),               "❌ Project folder not found in Drive"
assert (proj / "data").exists(),    "❌ data/ sub-folder missing"
assert (proj / "src").exists(),     "❌ src/ sub-folder missing"

# create new links now that the path is verified
os.symlink(proj / "data", "/content/data")
os.symlink(proj / "src",  "/content/src")

# switch into src and make it importable
sys.path.append("/content/src")
os.chdir("/content/src")

print("✅ Links rebuilt. Current dir:", os.getcwd())
print("Files here:", glob.glob('*'))


Mounted at /content/drive
✅ Links rebuilt. Current dir: /content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation/src
Files here: ['main.py', '__pycache__', 'model.py', 'dataset.py', 'train.py', 'H:']


In [26]:
!python train.py


/usr/local/lib/python3.11/dist-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-08-07 15:16:22.867113: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754579782.887577    9635 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754579782.894207    9635 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754579782.910270    9635 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754579782.910298    9635 computation_pl

In [27]:
import os
print(os.path.getsize("model_weights.pth")/1e6, "MB")


FileNotFoundError: [Errno 2] No such file or directory: 'model_weights.pth'

In [28]:
!grep -n "torch.save" /content/src/train.py


86:        torch.save(model.state_dict(), "/content/model_weights.pth")  # Adjust path for Colab


In [30]:
import os

path = "/content/model_weights.pth"   # where train.py saved the file
print("Exists?", os.path.exists(path))
if os.path.exists(path):
    print("Size (MB):", round(os.path.getsize(path) / 1e6, 2))


Exists? True
Size (MB): 486.1


In [31]:
import shutil, os, time

src = "/content/model_weights.pth"          # the file we just verified
dst_dir = "/content/drive/MyDrive/model_ckpts"  # any folder we would like in Drive
os.makedirs(dst_dir, exist_ok=True)

timestamp = time.strftime("%Y%m%d_%H%M%S")
dst = f"{dst_dir}/model_weights_{timestamp}.pth"

shutil.copy(src, dst)
print("✅ Checkpoint saved to Drive at:", dst)


✅ Checkpoint saved to Drive at: /content/drive/MyDrive/model_ckpts/model_weights_20250807_152952.pth


In [32]:
import shutil, pathlib

# Paths ─ adjust only if your folder names differ
src  = pathlib.Path("/content/drive/MyDrive/model_ckpts/model_weights_20250807_152952.pth")
dst_dir = pathlib.Path("/content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation")
dst = dst_dir / src.name         # keeps the same filename

# Create the destination folder if it’s missing
dst_dir.mkdir(parents=True, exist_ok=True)

# Move the file (change to shutil.copy if you prefer a duplicate)
shutil.move(src, dst)
print("✅ Moved to:", dst)


✅ Moved to: /content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation/model_weights_20250807_152952.pth


In [1]:
# Utility cell ─ run this whenever you need to confirm that
# 1) Google Drive is mounted, and
# 2) the mount is actually reachable.

from google.colab import drive
import os, time, subprocess

MOUNTPOINT = "/content/drive"

def colab_drive_status(mountpoint=MOUNTPOINT):
    status = {
        "is_dir": os.path.isdir(mountpoint),
        "is_mounted": False,
        "can_write": False,
        "latency_ms": None,
    }

    # ── 1 ▸ check /proc/mounts to see if FUSE drive is listed
    try:
        with open("/proc/mounts") as f:
            status["is_mounted"] = any(
                mountpoint in line and "fuse" in line for line in f
            )
    except FileNotFoundError:
        pass  # unlikely inside Colab

    # ── 2 ▸ simple I/O test (write + delete a tiny file)
    test_path = os.path.join(mountpoint, ".colab_drive_test")
    t0 = time.time()
    try:
        with open(test_path, "w") as f:
            f.write("ok")
        os.remove(test_path)
        status["can_write"] = True
        status["latency_ms"] = round((time.time() - t0) * 1_000, 1)
    except (OSError, PermissionError):
        status["can_write"] = False

    return status


# ------------------------------------------------------------------
# MAIN CHECK
stat = colab_drive_status()
print(f"Drive directory exists  : {stat['is_dir']}")
print(f"Drive listed as mounted : {stat['is_mounted']}")
print(f"Write test passed       : {stat['can_write']}")
if stat["latency_ms"] is not None:
    print(f"Write latency           : {stat['latency_ms']} ms")

if not stat["is_mounted"]:
    print("\n❗ Drive is NOT mounted. Run this:")
    print("from google.colab import drive")
    print("drive.mount('/content/drive')")


Drive directory exists  : False
Drive listed as mounted : False
Write test passed       : False

❗ Drive is NOT mounted. Run this:
from google.colab import drive
drive.mount('/content/drive')


In [2]:
from google.colab import drive
drive.mount('/content/drive')       # follow the authentication prompt


Mounted at /content/drive


In [3]:
import os, sys, shutil, glob
from pathlib import Path

proj = Path("/content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation")

# remove stale links if they exist
for link in ("/content/data", "/content/src"):
    if os.path.islink(link) or os.path.exists(link):
        shutil.rmtree(link, ignore_errors=True)

# create fresh links
os.symlink(proj / "data", "/content/data")
os.symlink(proj / "src",  "/content/src")

# switch into src and make it importable
sys.path.append("/content/src")
os.chdir("/content/src")
print("✅ Links rebuilt and Drive is active.")
print("Files:", glob.glob('*')[:10])


✅ Links rebuilt and Drive is active.
Files: ['main.py', '__pycache__', 'model.py', 'dataset.py', 'H:', 'train.py']


In [4]:
!python train.py

Traceback (most recent call last):
  File "/content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation/src/train.py", line 5, in <module>
    from dotenv import load_dotenv
ModuleNotFoundError: No module named 'dotenv'


In [5]:
!pip install --quiet python-dotenv==1.1.0

In [6]:
!python train.py

/usr/local/lib/python3.11/dist-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-08-07 17:11:18.993110: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754586679.012845    3440 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754586679.018878    3440 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754586679.034402    3440 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754586679.034426    3440 computation_pl

In [7]:
from PIL import Image
test_path = "/content/data/images/train/ISIC_7521364.jpg"
try:
  Image.open(test_path).verify()
  print("Image opened fine - Drive connection just glitched")
except Exception as e:
  print("Still failing:", e)

Image opened fine - Drive connection just glitched


In [8]:
# Unmount (ignore errors), then mount fresh
!fusermount -u /content/drive || true
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Re-establish shortcuts
import os, shutil, sys
proj = "/content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation"

for link in ("/content/data", "/content/src"):
    if os.path.islink(link) or os.path.exists(link):
        shutil.rmtree(link, ignore_errors=True)

os.symlink(f"{proj}/data", "/content/data")
os.symlink(f"{proj}/src",  "/content/src")

sys.path.append("/content/src")
print("✅ Drive remounted and links rebuilt")


fusermount: failed to unmount /content/drive: Device or resource busy
Mounted at /content/drive


FileExistsError: [Errno 17] File exists: '/content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation/data' -> '/content/data'

In [10]:
import os, shutil

for link in ("/content/data", "/content/src"):
    if os.path.islink(link):          # a dangling link
        os.unlink(link)
    elif os.path.exists(link):        # a real folder left behind
        shutil.rmtree(link, ignore_errors=True)

print("✅ Old links/folders cleared")


✅ Old links/folders cleared


In [11]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)   # you’ll see “Mounted at /content/drive”

import os, sys, glob
from pathlib import Path

proj = Path("/content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation")

# sanity-check the real folders exist
assert (proj / "data").exists(), "❌ data folder missing in Drive project"
assert (proj / "src").exists(),  "❌ src folder missing in Drive project"

# make brand-new links
os.symlink(proj / "data", "/content/data")
os.symlink(proj / "src",  "/content/src")

# activate the source directory
sys.path.append("/content/src")
os.chdir("/content/src")

print("✅ Links rebuilt. Current dir:", os.getcwd())
print("Files:", glob.glob('*')[:10])


Mounted at /content/drive
✅ Links rebuilt. Current dir: /content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation/src
Files: ['main.py', '__pycache__', 'model.py', 'dataset.py', 'H:', 'train.py']


In [ ]:
import shutil, pathlib, glob, os, time
local_dir = pathlib.Path("/content/chache_train_images")
if not local_dir.exists():
  t0 = time.time()
  print("First-time copy of images to local disk...")
  local_dir.mkdir(parents=True)
  for src in glob.glob("/content/data/images/train/*.jpg"):
    shutil.copy(src, local_dir)
  print(f"Copied {len(list(local_dir.glob('*.jpg'))):,} images "f"in {round(time.time()-t0, 1)} s")

First-time copy of images to local disk...


In [1]:
import pathlib, os, math, time

LOCAL_DIR = pathlib.Path("/content/cache_train_images")   # adjust if you chose another folder

t0 = time.time()
if not LOCAL_DIR.exists():
    raise FileNotFoundError(f"❌  Folder {LOCAL_DIR} does not exist")

jpg_files = list(LOCAL_DIR.glob("*.jpg"))
num_files = len(jpg_files)

total_bytes = sum(f.stat().st_size for f in jpg_files)
total_gb = total_bytes / 1_073_741_824     # bytes → GB (2^30)

print("✅ Local cache status")
print(f"   Images counted : {num_files:,}")
print(f"   Total size     : {total_gb:.2f} GB")
print(f"   Elapsed        : {round(time.time()-t0, 2)} s")


FileNotFoundError: ❌  Folder /content/cache_train_images does not exist

In [2]:
import pathlib, glob, shutil, time, os

# A. Search for any folder under /content that holds ≥30,000 jpgs
candidates = []
for p in pathlib.Path("/content").rglob("*.jpg"):
    # count quickly but stop after first match per parent dir
    parent = p.parent
    if parent not in candidates:
        if len(list(parent.glob("*.jpg"))) > 30_000:
            candidates.append(parent)
            break                # one good hit is enough

if candidates:
    LOCAL_DIR = candidates[0]
    print(f"✅ Found existing local cache: {LOCAL_DIR}")
else:
    # B. Nothing found ─ create folder and copy images now
    LOCAL_DIR = pathlib.Path("/content/cache_train_images")
    LOCAL_DIR.mkdir(parents=True, exist_ok=True)
    t0 = time.time()
    src_files = glob.glob("/content/data/images/train/*.jpg")
    print(f"⏳ Copying {len(src_files):,} images -> {LOCAL_DIR} …")
    for src in src_files:
        shutil.copy(src, LOCAL_DIR)
    print(f"✅ Copy complete in {round(time.time()-t0,1)} s")

# quick summary
num_jpg = len(list(LOCAL_DIR.glob("*.jpg")))
size_gb = sum(f.stat().st_size for f in LOCAL_DIR.glob("*.jpg")) / 1_073_741_824
print(f"Images counted : {num_jpg:,}")
print(f"Total size     : {size_gb:.2f} GB")


⏳ Copying 0 images -> /content/cache_train_images …
✅ Copy complete in 0.0 s
Images counted : 0
Total size     : 0.00 GB


In [3]:
import glob, os
src_pattern = "/content/data/images/train/*.jpg"
found = glob.glob(src_pattern)
print("JPEGs found at source:", len(found))
if len(found) < 10:
    # show a few paths for inspection
    print("Sample paths:", found[:5])


JPEGs found at source: 0
Sample paths: []


In [5]:
import os, glob, textwrap

def tree(path, depth=2):
    for root, dirs, files in os.walk(path):
        level = root.replace(path, '').count(os.sep)
        if level > depth:                 # stop after <depth> levels
            continue
        indent = ' ' * 3 * level
        print(f"{indent}{os.path.basename(root)}/")
        for d in dirs:
            print(f"{indent}   {d}/")
        break                             # don't descend further in this loop

tree("/content/data", depth=3)


In [6]:
import os, glob, textwrap

def tree(path, depth=2):
    for root, dirs, files in os.walk(path):
        level = root.replace(path, '').count(os.sep)
        if level > depth:                 # stop after <depth> levels
            continue
        indent = ' ' * 3 * level
        print(f"{indent}{os.path.basename(root)}/")
        for d in dirs:
            print(f"{indent}   {d}/")
        break                             # don't descend further in this loop

tree("/content/data", depth=3)


In [7]:
import glob, os
CANDIDATE = "/content/data/train/*.jpg"   # adjust if you saw a different path
files = glob.glob(CANDIDATE)
print("JPEGs here:", len(files))
print("Sample:", files[:3])


JPEGs here: 0
Sample: []


In [8]:
import os, glob, textwrap

def tree(path, depth=2):
    for root, dirs, files in os.walk(path):
        level = root.replace(path, '').count(os.sep)
        if level > depth:                 # stop after <depth> levels
            continue
        indent = ' ' * 3 * level
        print(f"{indent}{os.path.basename(root)}/")
        for d in dirs:
            print(f"{indent}   {d}/")
        break                             # don't descend further in this loop

tree("/content/data", depth=3)


In [9]:
from google.colab import drive
import os, time

mountpoint = "/content/drive"

print("Exists :", os.path.isdir(mountpoint))
print("Mounted:", any(mountpoint in l and "fuse" in l
                     for l in open("/proc/mounts")))


Exists : False
Mounted: False


In [10]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)   # follow the browser prompt


Mounted at /content/drive


In [11]:
import os, shutil, sys
proj = "/content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation"

for link in ("/content/data", "/content/src"):
    if os.path.islink(link) or os.path.exists(link):
        shutil.rmtree(link, ignore_errors=True)      # remove stale link/folder

os.symlink(f"{proj}/data", "/content/data")
os.symlink(f"{proj}/src",  "/content/src")

sys.path.append("/content/src")
print("✅ Links rebuilt")


✅ Links rebuilt


In [12]:
import glob, os
print("Images:", len(glob.glob("/content/data/images/train/*.jpg")))


Images: 0


In [13]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)   # click the link → copy code → paste


Mounted at /content/drive


In [14]:
import os, shutil, sys

proj = "/content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation"

# remove any stale links or folders
for link in ("/content/data", "/content/src"):
    if os.path.islink(link):
        os.unlink(link)
    elif os.path.exists(link):
        shutil.rmtree(link, ignore_errors=True)

# create fresh links that point into Drive
os.symlink(f"{proj}/data", "/content/data")
os.symlink(f"{proj}/src",  "/content/src")

sys.path.append("/content/src")   # make imports work
print("✅ Links rebuilt")


✅ Links rebuilt


In [15]:
import glob, os

img_pattern = "/content/data/images/train/*.jpg"
img_count   = len(glob.glob(img_pattern))

print("JPEGs now visible:", img_count)
print("Example path:", glob.glob(img_pattern)[:3])


JPEGs now visible: 33126
Example path: ['/content/data/images/train/ISIC_9713243.jpg', '/content/data/images/train/ISIC_9712403.jpg', '/content/data/images/train/ISIC_9711911.jpg']


In [18]:
!python train.py --image_dir /content/data/images/train


python3: can't open file '/content/train.py': [Errno 2] No such file or directory


In [19]:
!python /content/src/train.py --image_dir /content/data/images/train

Traceback (most recent call last):
  File "/content/src/train.py", line 5, in <module>
    from dotenv import load_dotenv
ModuleNotFoundError: No module named 'dotenv'


In [21]:
!pip install --quiet python-dotenv==1.0.0

In [22]:
!python /content/src/train.py --image_dir /content/data/images/train

2025-08-07 18:13:56.943164: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754590437.010888    7364 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754590437.032861    7364 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754590437.085667    7364 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754590437.085746    7364 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754590437.085756    7364 computation_placer.cc:177] computation placer alr

In [23]:
from dotenv import load_dotenv
print("✅ python-dotenv is available")

✅ python-dotenv is available


In [ ]:
%cd /content/src
!python train.py                       # no --image_dir flag


/content/drive/MyDrive/A Multimodal Generative AI System for Skin Lesion Diagnosis and Explanation/src
/usr/local/lib/python3.11/dist-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-08-07 18:15:21.457158: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754590521.480397    7819 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754590521.487278    7819 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754590521.505371    7819 computation_placer.cc:177] computation placer already registered. Please check linkage